In [ ]:
!unzip masked_validation_dataset.zip

In [19]:
!pip install datasets wandb evaluate accelerate -qU
!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/pytorch/text-classification/run_glue.py

--2024-03-27 13:40:27--  https://raw.githubusercontent.com/huggingface/transformers/master/examples/pytorch/text-classification/run_glue.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28440 (28K) [text/plain]
Saving to: ‘run_glue.py.1’

run_glue.py.1       100%[===================>]  27.77K  --.-KB/s    in 0.001s  

2024-03-27 13:40:27 (47.5 MB/s) - ‘run_glue.py.1’ saved [28440/28440]



In [20]:
!pip install wandb

In [1]:
from transformers import BertTokenizer, BertForMaskedLM
from datasets import Dataset
import torch

2024-03-27 13:28:12.539020: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
ds = Dataset.load_from_disk("masked_training_dataset")
ds_eval = Dataset.load_from_disk("masked_validation_dataset")

In [4]:
tokenized_lengths = [len(tokenizer.encode(s)) for s in ds_eval['text']]

# Find the maximum length
max_length = max(tokenized_lengths)
max_length

35

In [5]:
inputs = tokenizer(ds['text'], return_tensors='pt', max_length=35, truncation=True, padding='max_length')
inputs_eval= tokenizer(ds['text'], return_tensors='pt', max_length=35, truncation=True, padding='max_length')

In [6]:
inputs['labels'] = inputs.input_ids.detach().clone()
inputs_eval['labels'] = inputs.input_ids.detach().clone()

In [7]:
labels = tokenizer(ds["label"], add_special_tokens=False)
labels = [item for sublist in labels['input_ids'] for item in sublist]

labels_eval = tokenizer(ds_eval["label"], add_special_tokens=False)
labels_eval = [item for sublist in labels_eval['input_ids'] for item in sublist]

In [8]:
for tensor in inputs['labels']:
    for n, token in enumerate(tensor):
        if token.item() == tokenizer.mask_token_id:
            tensor[n] = torch.tensor(labels.pop(0))

for tensor_eval in inputs_eval['labels']:
    for n, token in enumerate(tensor):
        if token.item() == tokenizer.mask_token_id:
            tensor_eval[n] = torch.tensor(labels_eval.pop(0))

In [9]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [10]:
dataset_train = CustomDataset(inputs)
dataset_eval = CustomDataset(inputs_eval)

In [27]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir='out',
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    num_train_epochs=2,
    evaluate_during_training_steps=100
    report_to=None
)


TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluate_during_training'

In [23]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset_train,
    eval_dataset= dataset_eval
    
)

In [24]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: Currently logged in as: treuerpedro (temptue). Use `wandb login --relogin` to force relogin


/tmp/ipykernel_1706/2775228074.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss


TrainOutput(global_step=164, training_loss=0.5273202105266291, metrics={'train_runtime': 64.8977, 'train_samples_per_second': 321.861, 'train_steps_per_second': 2.527, 'total_flos': 375827684652000.0, 'train_loss': 0.5273202105266291, 'epoch': 2.0})